In [6]:
# Jupyter-ready: Insert one sales invoice item into the ONLY open form (is_empty=True),
# auto-computing item_number, validating VAT math, and preserving MongoDB Extended JSON.

import os, json, uuid, sys
from datetime import datetime, timezone
from dotenv import load_dotenv
from pymongo import MongoClient, ReturnDocument
from bson import json_util as bsonju

# -----------------------
# Configuration & Connect
# -----------------------
load_dotenv()
uri = os.getenv("MONGODB_URI")
db_name = os.getenv("MONGODB_DATABASE")
if not uri or not db_name:
    raise RuntimeError("Missing MONGODB_URI or MONGODB_DATABASE in environment.")

client = MongoClient(uri, serverSelectionTimeoutMS=5000)
col = client[db_name]["sales_invoices"]

def _err(reason, message, status=400, extra=None):
    payload = {"ok": False, "reason": reason, "message": message}
    if extra: payload.update(extra)
    print(json.dumps(payload, indent=2))
    sys.exit(1)

# -----------------------
# Actual payload (no item_number; computed here)
# -----------------------
payload = {
    "receipt_number": "1234",
    "customer_name": "Feret",
    "type": "fuel",             # "fuel" | "lubricant"
    "vatable_sales": 1785.71,
    "vat_amount": 214.29,
    "total_amount": 2000.00,
}

# -----------------------
# Lightweight validation
# -----------------------
ALLOWED_TYPES = {"fuel", "lubricant"}

if payload.get("type") not in ALLOWED_TYPES:
    _err("validation_error", f"type must be one of {sorted(ALLOWED_TYPES)}", extra={"got": payload.get("type")})

try:
    vs = float(payload["vatable_sales"])
    va = float(payload["vat_amount"])
    ta = float(payload["total_amount"])
except Exception:
    _err("validation_error", "vatable_sales, vat_amount, total_amount must be numbers")

# Check VAT math within ±0.01 tolerance
if abs((vs + va) - ta) > 0.01:
    _err("vat_mismatch", "vatable_sales + vat_amount must equal total_amount (±0.01)",
         extra={"vatable_sales": vs, "vat_amount": va, "total_amount": ta})

# -----------------------
# Locate exactly one open form
# -----------------------
open_forms = list(col.find({"is_empty": True}, {"_id": 1, "id": 1, "items.item_number": 1, "items.receipt_number": 1}).limit(2))
if len(open_forms) != 1:
    _err("need_exactly_one_open_form", "Expected exactly one document with is_empty=True", extra={"count": len(open_forms)})

doc = open_forms[0]

# Optional: ensure receipt_number uniqueness within the open form
existing_items = doc.get("items") or []
if any((it.get("receipt_number") == payload["receipt_number"]) for it in existing_items):
    _err("duplicate_receipt_number", "receipt_number already exists in the open form",
         extra={"receipt_number": payload["receipt_number"], "document_id": doc.get("id")})

# Compute next item_number from existing numeric item_numbers
def _parse_num(s):
    try:
        return int(str(s).strip())
    except Exception:
        return None

nums = []
for it in existing_items:
    n = _parse_num(it.get("item_number"))
    if n is not None:
        nums.append(n)

# -----------------------
# Construct new item
# -----------------------
now = datetime.now(timezone.utc)
new_item_base = {
    "id": str(uuid.uuid4()),
    "receipt_number": payload["receipt_number"],
    "customer_name": payload["customer_name"],
    "type": payload["type"],
    "vatable_sales": vs,
    "vat_amount": va,
    "total_amount": round(ta, 2),
    "created": now,
    "updated": now,
}

# -----------------------
# Concurrent-safe-ish insert with small retry loop
#   - Recomputes next_item_number each attempt
#   - Adds a guard that no item with the computed item_number exists at update time
# -----------------------
MAX_RETRIES = 3
attempt = 0
updated_doc = None

while attempt < MAX_RETRIES and updated_doc is None:
    attempt += 1

    # Re-read items to compute next_item_number fresh each attempt
    fresh = col.find_one({"_id": doc["_id"], "is_empty": True}, {"_id": 1, "id": 1, "items.item_number": 1})
    if not fresh:
        _err("open_form_changed", "Open form is no longer available or not is_empty=True")

    fresh_items = fresh.get("items") or []
    fresh_nums = []
    for it in fresh_items:
        n = _parse_num(it.get("item_number"))
        if n is not None:
            fresh_nums.append(n)

    next_item_number_int = (max(fresh_nums) + 1) if fresh_nums else 1
    next_item_number = str(next_item_number_int)

    # Guard: avoid duplicate item_number at update time
    filter_q = {
        "_id": doc["_id"],
        "is_empty": True,
        "items.item_number": {"$ne": next_item_number},  # best-effort guard
    }

    new_item = {**new_item_base, "item_number": next_item_number}

    updated_doc = col.find_one_and_update(
        filter_q,
        {"$push": {"items": new_item}},
        return_document=ReturnDocument.AFTER,
    )

# -----------------------
# Finalize / Output
# -----------------------
if updated_doc is None:
    _err("conflict", "Failed to insert after retries (likely concurrent inserts). Try again.")

added = next((it for it in (updated_doc.get("items") or []) if it.get("id") == new_item_base["id"]), None)
if not added:
    _err("insert_unknown_state", "Insert may have raced; added item not found on document.", extra={"document_id": updated_doc.get("id")})

print(bsonju.dumps({
    "ok": True,
    "message": "item added to open form (is_empty unchanged)",
    "document_id": updated_doc.get("id"),
    "item": added
}, indent=2))


{
  "ok": true,
  "message": "item added to open form (is_empty unchanged)",
  "document_id": "6b4a1918-5d36-44b6-be53-1d2cbba5b64b",
  "item": {
    "id": "7c679dfe-48f0-4866-8b3e-468ca2295fe2",
    "receipt_number": "1234",
    "customer_name": "Feret",
    "type": "fuel",
    "vatable_sales": 1785.71,
    "vat_amount": 214.29,
    "total_amount": 2000.0,
    "created": {
      "$date": 1760467280156
    },
    "updated": {
      "$date": 1760467280156
    },
    "item_number": "4"
  }
}
